## Experiments: Comparing RSAST with other methods

In [73]:
#configure directory to import sast libraries
import sys 
import os 
#add sast library path
file_path = os.path.expanduser("~/random_sast/sast")
sys.path.append(file_path)
#add cd_diagram library path
file_path = os.path.expanduser("~/random_sast/cd_diagram")
sys.path.append(file_path)

#sys.path

In [74]:
from sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd
from sktime.transformations.panel.shapelet_transform import ShapeletTransform
import matplotlib.pyplot as plt
import os

hyperparameter='ACF&PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=100'

### Reading Datasets

In [75]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)

        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)
df_other_methods=df_other_methods[['filename','folds:','0']]
df_other_methods['method']=df_other_methods['filename'].str.split('_').str[0]
df_other_methods
df_other_methods=df_other_methods.rename(columns={'folds:':'dataset','0':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]



In [76]:
df_other_methods['method'].unique()

array(['SAST', 'ROCKET', 'HIVE-COTEv1', 'BOSS', 'RISE', 'TSF',
       'InceptionTime', 'TS-CHIEF', 'S-BOSS', 'STC', 'ResNet',
       'ProximityForest', 'WEASEL', 'Catch22', 'cBOSS'], dtype=object)

In [77]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsRsast'
#directory = os.getcwd()+'/results_accuracy_per_ds'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.startswith("df_overall_tunning") and filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        df['rpoint']=df['rpoint'].astype(str)
        df['nb_per_class']=df['nb_per_class'].astype(str)
        
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)
df_result.columns

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method'],
      dtype='object')

In [78]:
df_result=df_result[['accuracy','dataset_name','classifier_name','time']]

df_result=df_result[df_result["classifier_name"]==hyperparameter]
df_result.dataset_name.unique()


array(['Chinatown', 'SmoothSubspace', 'SonyAIBORobotSurface2',
       'ItalyPowerDemand', 'GunPoint', 'Coffee', 'SonyAIBORobotSurface1',
       'DistalPhalanxOutlineCorrect'], dtype=object)

In [79]:

df_other_methods=df_other_methods.rename(columns={"dataset":"dataset_name","score":"accuracy","method":"classifier_name"})
df_other_methods = df_other_methods[df_other_methods['dataset_name'].isin(df_result.dataset_name.values)]
df_other_methods.columns

Index(['dataset_name', 'accuracy', 'classifier_name'], dtype='object')

In [80]:
# concatenate the two dataframes
concatenated_df = pd.concat([df_other_methods, df_result])

# reset the index of the concatenated dataframe
concatenated_df = concatenated_df.reset_index(drop=True)


In [81]:
concatenated_df['classifier_name'].unique()

array(['SAST', 'ROCKET', 'HIVE-COTEv1', 'BOSS', 'RISE', 'TSF',
       'InceptionTime', 'TS-CHIEF', 'S-BOSS', 'STC', 'ResNet',
       'ProximityForest', 'WEASEL', 'Catch22', 'cBOSS',
       'ACF&PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=100'],
      dtype=object)

### Overall analysis

#### Accuracy: RSAST vs Other methods

In [82]:
# create a pivot table with the mean of score by hyperparameter
score_by_method=pd.pivot_table(concatenated_df, values='accuracy', index=['classifier_name'],columns=['dataset_name'], aggfunc='mean')



In [83]:
t_score_by_method=score_by_method.transpose()
t_score_by_method
#t_score_by_method[[hyperparameter,"STC","ROCKET"]]


classifier_name,ACF&PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=100,BOSS,Catch22,HIVE-COTEv1,InceptionTime,ProximityForest,RISE,ROCKET,ResNet,S-BOSS,SAST,STC,TS-CHIEF,TSF,WEASEL,cBOSS
dataset_name,,,,,,,,,,,,,,,,
Chinatown,0.979592,0.766764,0.930435,0.979592,0.988338,0.971014,0.959184,0.982507,0.976812,0.784257,0.96,0.967930,0.968116,0.982507,0.956268,0.950725
Coffee,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00,1.000000,1.000000,1.000000,1.000000,1.000000
DistalPhalanxOutlineCorrect,0.735507,0.739130,0.800725,0.778986,0.793478,0.804348,0.775362,0.771739,0.764493,0.739130,0.74,0.771739,0.757246,0.764493,0.775362,0.731884
GunPoint,0.984444,1.000000,0.946667,0.993333,1.000000,1.000000,0.980000,1.000000,0.993333,1.000000,0.97,0.993333,1.000000,0.960000,1.000000,1.000000
ItalyPowerDemand,0.949142,0.908649,0.896016,0.962099,0.965986,0.965986,0.953353,0.968902,0.965015,0.908649,0.96,0.955296,0.965015,0.966958,0.955296,0.923226
SmoothSubspace,0.908889,0.393333,0.866667,0.980000,0.986667,1.000000,0.873333,0.980000,1.000000,0.393333,0.91,0.933333,1.000000,0.986667,0.833333,0.520000
SonyAIBORobotSurface1,0.807543,0.632280,0.851913,0.753744,0.878536,0.836938,0.821963,0.921797,0.956739,0.668885,0.76,0.752080,0.831947,0.808652,0.845258,0.537438
SonyAIBORobotSurface2,0.912207,0.859391,0.922350,0.935992,0.951731,0.898216,0.910808,0.916055,0.973767,0.876180,0.85,0.943337,0.896118,0.834208,0.947534,0.907660


In [84]:
for method in t_score_by_method.columns.drop(hyperparameter):
    y=t_score_by_method[hyperparameter]
    x=t_score_by_method[method]

    # set the style of the plot
    plt.style.use('seaborn')

    # create the plot
    fig, ax = plt.subplots(figsize=(8, 6))

    # plot the line
    ax.plot([0,1], [0,1])

    # plot the points
    ax.scatter(x, y, color='blue', edgecolor='blue', s=30, zorder=10)

    # add labels and title
    ax.set_ylabel('RSAST')
    ax.set_xlabel(method)
    ax.set_title('Accuracy')


    # set the background color
    ax.set_facecolor('lightgrey')
    plt.savefig('images_one_vs_one_comparison/RSASTvs'+method+'.png')


/tmp/ipykernel_5379/3124073668.py:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')
/tmp/ipykernel_5379/3124073668.py:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')
/tmp/ipykernel_5379/3124073668.py:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.s

In [85]:
# create a pivot table with the mean of time by hyperparameter
time_per_method=pd.pivot_table(df_result, values='time', columns=['classifier_name'],index=['dataset_name'], aggfunc='mean')/60
time_per_method

classifier_name,ACF&PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=100
dataset_name,
Chinatown,0.002399
Coffee,8.096969
DistalPhalanxOutlineCorrect,21.882810
GunPoint,3.756236
ItalyPowerDemand,0.021976
SmoothSubspace,0.024465
SonyAIBORobotSurface1,0.030248
SonyAIBORobotSurface2,0.062513


In [86]:
# create a pivot table with the variation of score by hyperparameter
pd.pivot_table(concatenated_df, values='accuracy', index=['classifier_name'],columns=['dataset_name'], aggfunc='var')


dataset_name,Chinatown,GunPoint,ItalyPowerDemand,SmoothSubspace,SonyAIBORobotSurface1,SonyAIBORobotSurface2
classifier_name,,,,,,
ACF&PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=100,0.000008,0.000015,0.000032,0.000637,0.000059,0.000003


#### Generate CD Diagram comparison

In [87]:

from cd_function import *
df_perf=pd.pivot_table(concatenated_df, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
draw_cd_diagram(df_perf, labels=True, title="Overall Comparison Other Methods", name_file='images_cd_diagram/cd-diagram_overall_comparison_other_method.png')


/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


In [88]:
df_perf["classifier_name"].unique()

array(['ACF&PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=100',
       'BOSS', 'Catch22', 'HIVE-COTEv1', 'InceptionTime',
       'ProximityForest', 'RISE', 'ROCKET', 'ResNet', 'S-BOSS', 'SAST',
       'STC', 'TS-CHIEF', 'TSF', 'WEASEL', 'cBOSS'], dtype=object)

In [89]:
_, average_ranks, _ = wilcoxon_holm(df_perf=df_perf)
min_rank= min(average_ranks)
average_ranks=pd.DataFrame(average_ranks)
average_ranks

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


,0
BOSS,12.6875
S-BOSS,12.3125
cBOSS,11.8750
SAST,10.9375
ACF&PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=100,10.0625
Catch22,9.8125
RISE,9.6250
STC,8.6875
TSF,8.3750
WEASEL,7.6250
